In [3]:
#SCIB scripts

import os
os.chdir('/data/kkovacs/Python')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
import scanpy as sc
import copy
import torch
import scarches as sca
from scarches.dataset.trvae.data_handling import remove_sparsity
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import gdown
import sankey
import anndata
import pandas as pd
from pySankey.sankey import sankey
from matplotlib.pyplot import rc_context
import scib
#unable to import SAUCIE due to version issues

import warnings
warnings.filterwarnings("ignore")
#Doing this only output convenience 

In [3]:
adata = sc.read("/data/kkovacs/Python/Data/Full_corrected_counts_and_genes_post_filtering.h5ad")
#Using the healthy controls to test the scores
adata.X.data = adata.layers['raw'].data.copy()

#Creating new metadata in accordance with SCIB variables
adata.obs['cluster'] = adata.obs['clusters_1']
adata.obs['celltype'] = adata.obs['clust']
adata.obs['batch'] = adata.obs['study']
adata.layers['counts'] = adata.layers['raw']

print(adata.X.data)
adata.shape

[3. 3. 2. 2. 1. 1. 3. ... 1. 3. 8. 1. 1. 1. 1.]


(174977, 43519)

In [9]:
adata.var['highly_variable_2k'] = adata.var['highly_variable']

adata.var['highly_variable_2k'].value_counts()

adata.layers['raw'] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata
adata.layers['norm'] = adata.X.copy()

In [10]:
sc.pp.highly_variable_genes(adata, batch_key="study",min_mean=0.03, flavor="cell_ranger",n_top_genes=6000)
adata.var['highly_variable'].value_counts()

highly_variable
False    37519
True      6000
Name: count, dtype: int64

In [24]:
#When the SCIB script is run, there were some issues where the replicated objects seemed linked through 
#the process and had created dupliacted results, when one had not be run through the process

#this script is just used to re-run the adata object and return to the correct point

def ReRunAdata(HVG,scaled):
	adata = sc.read("/data/kkovacs/Python/Data/MLCA_healthy.h5ad")
	adata = adata[:,HVG]
	adata.obs['cluster'] = adata.obs['clusters_1']
	adata.obs['celltype'] = adata.obs['clust']
	adata.obs['batch'] = adata.obs['study']
	if scaled == True:
		adata.layers['counts'] = adata.layers['raw']
		sc.pp.normalize_total(adata)
		sc.pp.scale(adata)
		sc.tl.pca(adata)
		sc.pp.neighbors(adata)
	else:
		adata.layers['counts'] = adata.layers['raw']
		sc.pp.normalize_total(adata)
		sc.tl.pca(adata)
		sc.pp.neighbors(adata)
	return(adata)

In [25]:
#This is the same as above but only for SCVI and SCANVI run throughs as they only require raw counts
def ReRunAdata_SC(HVG):
	adata = sc.read("/data/kkovacs/Python/Data/MLCA_healthy.h5ad")
	adata = adata[:,HVG]
	adata.obs['cluster'] = adata.obs['clusters_1']
	adata.obs['celltype'] = adata.obs['clust']
	adata.obs['batch'] = adata.obs['study']
	adata.layers['counts'] = adata.layers['raw']
	sc.tl.pca(adata)
	sc.pp.neighbors(adata)
	return(adata)

In [15]:
healthy.var['highly_variable'].value_counts()

highly_variable
False    41519
True      2000
Name: count, dtype: int64

In [18]:
adata.var['2k_highly_variable'] = adata.var['highly_variable']
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5,span=0.3,n_bins=20)
sc.pp.highly_variable_genes(adata,n_top_genes=6000)

adata.var['2k_highly_variable'].value_counts()
adata.var['highly_variable'].value_counts()

#CHANGE THESE HVG TO LISTS TO PUT IN CSV FILES

highly_variable
False    37519
True      6000
Name: count, dtype: int64

In [19]:
SixK = adata.var_names[adata.var['highly_variable']]
twoK = adata.var_names[adata.var['highly_variable_2k']]

#Trajectory was turned to false to prevent errors, was not used in HLCA either

def runScib_SCANVI():
	adata = ReRunAdata_SC(SixK)
	adata = adata[:,SixK]
	sc.tl.pca(adata)
	sc.pp.neighbors(adata)
	scib.me.cluster_optimal_resolution(adata, cluster_key="cluster",\
		label_key="batch")
	adata_int = scib.ig.scanvi(adata, batch="batch",labels = 'celltype')
	sc.pp.neighbors(adata_int, use_rep="X_emb")
	scib.me.cluster_optimal_resolution(adata_int, cluster_key="cluster", label_key="celltype")
	scib.pp.reduce_data(adata_int, n_top_genes=6000, batch_key="batch", use_rep = 'X_emb', pca=False, neighbors=True)
	adata = ReRunAdata_SC(SixK)
	score = scib.metrics.metrics(adata, adata_int, batch_key = 'batch', 
		label_key = 'celltype',embed='X_pca', cluster_key='cluster', cluster_nmi=None,
		ari_=True, nmi_=True, nmi_method='arithmetic', nmi_dir=True, 
		silhouette_=True, si_metric='euclidean', pcr_=True, cell_cycle_=True, 
		organism='mouse', hvg_score_=False, isolated_labels_=True, 
		isolated_labels_f1_=True, isolated_labels_asw_=True, 
		n_isolated=None, graph_conn_=True, trajectory_=False, 
		kBET_=True, lisi_graph_=True, ilisi_=True, clisi_=True, 
		subsample=0.5, n_cores=5, type_=None, verbose=Fals)
	adata = ReRunAdata_SC(twoK)
	scib.me.cluster_optimal_resolution(adata, cluster_key="cluster",\
		label_key="batch")
	adata.layers['counts'] = adata.layers['raw']
	adata_int = scib.ig.scanvi(adata, batch="batch",labels='celltype')
	sc.pp.neighbors(adata_int, use_rep="X_emb")
	scib.me.cluster_optimal_resolution(adata_int, cluster_key="cluster", label_key="celltype")
	scib.pp.reduce_data(adata_int, n_top_genes=2000, batch_key="batch", use_rep = 'X_emb', pca=False, neighbors=True)
	adata = ReRunAdata_SC(twoK)
	score_2k = scib.metrics.metrics(adata, adata_int, batch_key = 'batch', 
		label_key = 'celltype',embed='X_pca', cluster_key='cluster', cluster_nmi=None,
		ari_=True, nmi_=True, nmi_method='arithmetic', nmi_dir=True, 
		silhouette_=True, si_metric='euclidean', pcr_=True, cell_cycle_=True, 
		organism='mouse', hvg_score_=False, isolated_labels_=True, 
		isolated_labels_f1_=True, isolated_labels_asw_=True, 
		n_isolated=None, graph_conn_=True, trajectory_=False, 
		kBET_=True, lisi_graph_=True, ilisi_=True, clisi_=True, 
		subsample=0.5, n_cores=5, type_=None, verbose=False)
	score_2k.to_csv('/data/kkovacs/Python/score_scanvi_2k.csv')
	
	return(adata)

In [20]:

def runScib_SCVI():
	adata = ReRunAdata_SC(SixK)
	adata = adata[:,SixK]
	sc.tl.pca(adata)
	sc.pp.neighbors(adata)
	scib.me.cluster_optimal_resolution(adata, cluster_key="cluster",\
		label_key="batch")
	adata_int = scib.ig.scvi(adata, batch="batch")
	sc.pp.neighbors(adata_int, use_rep="X_emb")
	scib.me.cluster_optimal_resolution(adata_int, cluster_key="cluster", label_key="celltype")
	scib.pp.reduce_data(adata_int, n_top_genes=6000, batch_key="batch", use_rep = 'X_emb', pca=False, neighbors=True)
	adata = ReRunAdata_SC(SixK)
	score = scib.metrics.metrics(adata, adata_int, batch_key = 'batch', 
		label_key = 'celltype',embed='X_pca', cluster_key='cluster', cluster_nmi=None,
		ari_=True, nmi_=True, nmi_method='arithmetic', nmi_dir=True, 
		silhouette_=True, si_metric='euclidean', pcr_=True, cell_cycle_=True, 
		organism='mouse', hvg_score_=False, isolated_labels_=True, 
		isolated_labels_f1_=True, isolated_labels_asw_=True, 
		n_isolated=None, graph_conn_=True, trajectory_=False, 
		kBET_=True, lisi_graph_=True, ilisi_=True, clisi_=True, 
		subsample=0.5, n_cores=5, type_=None, verbose=False)
	score.to_csv('/data/kkovacs/Python/score_scvi_6k.csv')
	adata = ReRunAdata_SC(twoK)
	scib.me.cluster_optimal_resolution(adata, cluster_key="cluster",\
		label_key="batch")
	adata.layers['counts'] = adata.layers['raw']
	adata_int = scib.ig.scvi(adata, batch="batch")
	sc.pp.neighbors(adata_int, use_rep="X_emb")
	scib.me.cluster_optimal_resolution(adata_int, cluster_key="cluster", label_key="celltype")
	scib.pp.reduce_data(adata_int, n_top_genes=2000, batch_key="batch", use_rep = 'X_emb', pca=False, neighbors=True)
	adata = ReRunAdata_SC(twoK)
	score_2k = scib.metrics.metrics(adata, adata_int, batch_key = 'batch', 
		label_key = 'celltype',embed='X_pca', cluster_key='cluster', cluster_nmi=None,
		ari_=True, nmi_=True, nmi_method='arithmetic', nmi_dir=True, 
		silhouette_=True, si_metric='euclidean', pcr_=True, cell_cycle_=True, 
		organism='mouse', hvg_score_=False, isolated_labels_=True, 
		isolated_labels_f1_=True, isolated_labels_asw_=True, 
		n_isolated=None, graph_conn_=True, trajectory_=False, 
		kBET_=True, lisi_graph_=True, ilisi_=True, clisi_=True, 
		subsample=0.5, n_cores=5, type_=None, verbose=False)
	score_2k.to_csv('/data/kkovacs/Python/score_scvi_2k.csv')
	
	return adata

In [21]:

def runScib_Non_raw(method):
	adata = ReRunAdata(sixk,False)
	scib.me.cluster_optimal_resolution(adata, cluster_key="cluster",\
		label_key="batch")
	adata_int = getattr(scib.ig, method)(adata,batch='batch')
	adata_int.uns = adata.uns.copy()
	adata_int.obsp = adata.obsp.copy()
	adata_int.layers = adata.layers.copy()
	if 'X_emb' in adata_int.obsm:
		sc.pp.neighbors(adata_int, use_rep = 'X_emb')
		scib.me.cluster_optimal_resolution(adata_int, cluster_key="cluster", label_key="celltype")
		scib.pp.reduce_data(adata_int, n_top_genes=6000, batch_key="batch", use_rep = 'X_emb', pca=False, neighbors=True)
	else:
		sc.tl.pca(adata_int)
		sc.pp.neighbors(adata_int)
		scib.me.cluster_optimal_resolution(adata_int, cluster_key="cluster", label_key="celltype")
		scib.pp.reduce_data(adata_int, n_top_genes=6000, batch_key="batch", pca=True, neighbors=True)
	adata = ReRunAdata(sixk,False)
	score_2k = scib.metrics.metrics(adata, adata_int, batch_key = 'batch', 
		label_key = 'celltype',embed='X_pca', cluster_key='cluster', cluster_nmi=None,
		ari_=True, nmi_=True, nmi_method='arithmetic', nmi_dir=True, 
		silhouette_=True, si_metric='euclidean', pcr_=True, cell_cycle_=True, 
		organism='mouse', hvg_score_=False, isolated_labels_=True, 
		isolated_labels_f1_=True, isolated_labels_asw_=True, 
		n_isolated=None, graph_conn_=True, trajectory_=False, 
		kBET_=True, lisi_graph_=True, ilisi_=True, clisi_=True, 
		subsample=0.5, n_cores=5, type_=None, verbose=False)
	score_2k.to_csv('/data/kkovacs/Python/score_{}_6k_unscaled.csv'.format(method))
		#####
	adata = ReRunAdata(twoK,True)
	scib.me.cluster_optimal_resolution(adata, cluster_key="cluster",\
		label_key="batch")
	adata_int = getattr(scib.ig, method)(adata,batch='batch')
	adata_int.uns = adata.uns.copy()
	adata_int.obsp = adata.obsp.copy()
	adata_int.layers = adata.layers.copy()
	if 'X_emb' in adata_int.obsm:
		sc.pp.neighbors(adata_int, use_rep = 'X_emb')
		scib.me.cluster_optimal_resolution(adata_int, cluster_key="cluster", label_key="celltype")
		scib.pp.reduce_data(adata_int, n_top_genes=6000, batch_key="batch", use_rep = 'X_emb', pca=False, neighbors=True)
	else:
		sc.tl.pca(adata_int)
		sc.pp.neighbors(adata_int)
		scib.me.cluster_optimal_resolution(adata_int, cluster_key="cluster", label_key="celltype")
		scib.pp.reduce_data(adata_int, n_top_genes=6000, batch_key="batch", pca=True, neighbors=True)
	adata = ReRunAdata(twoK,True)
	score = scib.metrics.metrics(adata, adata_int, batch_key = 'batch', 
		label_key = 'celltype',embed='X_pca', cluster_key='cluster', cluster_nmi=None,
		ari_=True, nmi_=True, nmi_method='arithmetic', nmi_dir=True, 
		silhouette_=True, si_metric='euclidean', pcr_=True, cell_cycle_=True, 
		organism='mouse', hvg_score_=False, isolated_labels_=True, 
		isolated_labels_f1_=True, isolated_labels_asw_=True, 
		n_isolated=None, graph_conn_=True, trajectory_=False, 
		kBET_=True, lisi_graph_=True, ilisi_=True, clisi_=True, 
		subsample=0.5, n_cores=5, type_=None, verbose=False)
	score.to_csv('/data/kkovacs/Python/score_{}_2k_scaled.csv'.format(method))
	adata = ReRunAdata(twoK,False)
	scib.me.cluster_optimal_resolution(adata, cluster_key="cluster",\
		label_key="batch")
	adata_int = getattr(scib.ig, method)(adata,batch='batch')
	adata_int.uns = adata.uns.copy()
	adata_int.obsp = adata.obsp.copy()
	adata_int.layers = adata.layers.copy()
	if 'X_emb' in adata_int.obsm:
		sc.pp.neighbors(adata_int, use_rep = 'X_emb')
		scib.me.cluster_optimal_resolution(adata_int, cluster_key="cluster", label_key="celltype")
		scib.pp.reduce_data(adata_int, n_top_genes=2000, batch_key="batch", use_rep = 'X_emb', pca=False, neighbors=True)
	else:
		sc.tl.pca(adata_int)
		sc.pp.neighbors(adata_int)
		scib.me.cluster_optimal_resolution(adata_int, cluster_key="cluster", label_key="celltype")
		scib.pp.reduce_data(adata_int, n_top_genes=2000, batch_key="batch", pca=True, neighbors=True)
	adata = ReRunAdata(twoK,False)
	score_2k = scib.metrics.metrics(adata, adata_int, batch_key = 'batch', 
		label_key = 'celltype',embed='X_pca', cluster_key='cluster', cluster_nmi=None,
		ari_=True, nmi_=True, nmi_method='arithmetic', nmi_dir=True, 
		silhouette_=True, si_metric='euclidean', pcr_=True, cell_cycle_=True, 
		organism='mouse', hvg_score_=False, isolated_labels_=True, 
		isolated_labels_f1_=True, isolated_labels_asw_=True, 
		n_isolated=None, graph_conn_=True, trajectory_=False, 
		kBET_=True, lisi_graph_=True, ilisi_=True, clisi_=True, 
		subsample=0.5, n_cores=5, type_=None, verbose=False)
	score_2k.to_csv('/data/kkovacs/Python/score_{}_2k_unscaled.csv'.format(method))
	return adata

In [22]:
def runScib_Non_raw_scanorama(method):
	adata = ReRunAdata(SixK,True)
	scib.me.cluster_optimal_resolution(adata, cluster_key="cluster",\
		label_key="batch")
	adata_int = getattr(scib.ig, method)(adata,batch='batch')
	adata_int.uns = adata.uns.copy()
	adata_int.obsp = adata.obsp.copy()
	adata_int.layers = adata.layers.copy()
	if 'X_emb' in adata_int.obsm:
		sc.pp.neighbors(adata_int, use_rep = 'X_emb')
		scib.me.cluster_optimal_resolution(adata_int, cluster_key="cluster", label_key="celltype")
		scib.pp.reduce_data(adata_int, n_top_genes=6000, batch_key="batch", use_rep = 'X_emb', pca=False, neighbors=True)
	else:
		sc.tl.pca(adata_int)
		sc.pp.neighbors(adata_int)
		scib.me.cluster_optimal_resolution(adata_int, cluster_key="cluster", label_key="celltype")
		scib.pp.reduce_data(adata_int, n_top_genes=6000, batch_key="batch", pca=True, neighbors=True)
	adata = ReRunAdata(SixK,True)
	score = scib.metrics.metrics(adata, adata_int, batch_key = 'batch', 
		label_key = 'celltype',embed='X_pca', cluster_key='cluster', cluster_nmi=None,
		ari_=True, nmi_=True, nmi_method='arithmetic', nmi_dir=True, 
		silhouette_=True, si_metric='euclidean', pcr_=True, cell_cycle_=True, 
		organism='mouse', hvg_score_=False, isolated_labels_=True, 
		isolated_labels_f1_=True, isolated_labels_asw_=True, 
		n_isolated=None, graph_conn_=True, trajectory_=False, 
		kBET_=True, lisi_graph_=True, ilisi_=True, clisi_=True, 
		subsample=0.5, n_cores=5, type_=None, verbose=False)
	score.to_csv('/data/kkovacs/Python/score_{}_6k_scaled.csv'.format(method))
	adata = ReRunAdata(SixK,False)
	scib.me.cluster_optimal_resolution(adata, cluster_key="cluster",\
		label_key="batch")
	adata_int = getattr(scib.ig, method)(adata,batch='batch')
	adata_int.uns = adata.uns.copy()
	adata_int.obsp = adata.obsp.copy()
	adata_int.layers = adata.layers.copy()
	if 'X_emb' in adata_int.obsm:
		sc.pp.neighbors(adata_int, use_rep = 'X_emb')
		scib.me.cluster_optimal_resolution(adata_int, cluster_key="cluster", label_key="celltype")
		scib.pp.reduce_data(adata_int, n_top_genes=6000, batch_key="batch", use_rep = 'X_emb', pca=False, neighbors=True)
	else:
		sc.tl.pca(adata_int)
		sc.pp.neighbors(adata_int)
		scib.me.cluster_optimal_resolution(adata_int, cluster_key="cluster", label_key="celltype")
		scib.pp.reduce_data(adata_int, n_top_genes=6000, batch_key="batch", pca=True, neighbors=True)
	adata = ReRunAdata(SixK,False)
	score_2k = scib.metrics.metrics(adata, adata_int, batch_key = 'batch', 
		label_key = 'celltype',embed='X_pca', cluster_key='cluster', cluster_nmi=None,
		ari_=True, nmi_=True, nmi_method='arithmetic', nmi_dir=True, 
		silhouette_=True, si_metric='euclidean', pcr_=True, cell_cycle_=True, 
		organism='mouse', hvg_score_=False, isolated_labels_=True, 
		isolated_labels_f1_=True, isolated_labels_asw_=True, 
		n_isolated=None, graph_conn_=True, trajectory_=False, 
		kBET_=True, lisi_graph_=True, ilisi_=True, clisi_=True, 
		subsample=0.5, n_cores=5, type_=None, verbose=False)
	score_2k.to_csv('/data/kkovacs/Python/score_{}_6k_unscaled.csv'.format(method))
		#####
	adata = ReRunAdata(twoK,True)
	scib.me.cluster_optimal_resolution(adata, cluster_key="cluster",\
		label_key="batch")
	adata_int = getattr(scib.ig, method)(adata,batch='batch')
	adata_int.uns = adata.uns.copy()
	adata_int.obsp = adata.obsp.copy()
	adata_int.layers = adata.layers.copy()
	if 'X_emb' in adata_int.obsm:
		sc.pp.neighbors(adata_int, use_rep = 'X_emb')
		scib.me.cluster_optimal_resolution(adata_int, cluster_key="cluster", label_key="celltype")
		scib.pp.reduce_data(adata_int, n_top_genes=2000, batch_key="batch", use_rep = 'X_emb', pca=False, neighbors=True)
	else:
		sc.tl.pca(adata_int)
		sc.pp.neighbors(adata_int)
		scib.me.cluster_optimal_resolution(adata_int, cluster_key="cluster", label_key="celltype")
		scib.pp.reduce_data(adata_int, n_top_genes=2000, batch_key="batch", pca=True, neighbors=True)
	adata = ReRunAdata(twoK,True)
	score = scib.metrics.metrics(adata, adata_int, batch_key = 'batch', 
		label_key = 'celltype',embed='X_pca', cluster_key='cluster', cluster_nmi=None,
		ari_=True, nmi_=True, nmi_method='arithmetic', nmi_dir=True, 
		silhouette_=True, si_metric='euclidean', pcr_=True, cell_cycle_=True, 
		organism='mouse', hvg_score_=False, isolated_labels_=True, 
		isolated_labels_f1_=True, isolated_labels_asw_=True, 
		n_isolated=None, graph_conn_=True, trajectory_=False, 
		kBET_=True, lisi_graph_=True, ilisi_=True, clisi_=True, 
		subsample=0.5, n_cores=5, type_=None, verbose=False)
	score.to_csv('/data/kkovacs/Python/score_{}_2k_scaled.csv'.format(method))
	adata = ReRunAdata(twoK,False)
	scib.me.cluster_optimal_resolution(adata, cluster_key="cluster",\
		label_key="batch")
	adata_int = getattr(scib.ig, method)(adata,batch='batch')
	adata_int.uns = adata.uns.copy()
	adata_int.obsp = adata.obsp.copy()
	adata_int.layers = adata.layers.copy()
	if 'X_emb' in adata_int.obsm:
		sc.pp.neighbors(adata_int, use_rep = 'X_emb')
		scib.me.cluster_optimal_resolution(adata_int, cluster_key="cluster", label_key="celltype")
		scib.pp.reduce_data(adata_int, n_top_genes=2000, batch_key="batch", use_rep = 'X_emb', pca=False, neighbors=True)
	else:
		sc.tl.pca(adata_int)
		sc.pp.neighbors(adata_int)
		scib.me.cluster_optimal_resolution(adata_int, cluster_key="cluster", label_key="celltype")
		scib.pp.reduce_data(adata_int, n_top_genes=2000, batch_key="batch", pca=True, neighbors=True)
	adata = ReRunAdata(twoK,False)
	score_2k = scib.metrics.metrics(adata, adata_int, batch_key = 'batch', 
		label_key = 'celltype',embed='X_pca', cluster_key='cluster', cluster_nmi=None,
		ari_=True, nmi_=True, nmi_method='arithmetic', nmi_dir=True, 
		silhouette_=True, si_metric='euclidean', pcr_=True, cell_cycle_=True, 
		organism='mouse', hvg_score_=False, isolated_labels_=True, 
		isolated_labels_f1_=True, isolated_labels_asw_=True, 
		n_isolated=None, graph_conn_=True, trajectory_=False, 
		kBET_=True, lisi_graph_=True, ilisi_=True, clisi_=True, 
		subsample=0.5, n_cores=5, type_=None, verbose=False)
	score_2k.to_csv('/data/kkovacs/Python/score_{}_2k_unscaled.csv'.format(method))
	return adata

In [26]:
runScib_SCVI()

resolution: 0.1, nmi: 0.5062047430710064
resolution: 0.2, nmi: 0.5394980332770862
resolution: 0.3, nmi: 0.5550980109157683
resolution: 0.4, nmi: 0.565884647376148
resolution: 0.5, nmi: 0.5718045786861633
resolution: 0.6, nmi: 0.5739409380064459
resolution: 0.7, nmi: 0.5803614831855262
resolution: 0.8, nmi: 0.5811726329205968
resolution: 0.9, nmi: 0.5888874589183382
resolution: 1.0, nmi: 0.596875005891779
resolution: 1.1, nmi: 0.5936411336381535
resolution: 1.2, nmi: 0.5935711198208106
resolution: 1.3, nmi: 0.5922128427673292
resolution: 1.4, nmi: 0.5867140528453052
resolution: 1.5, nmi: 0.5864500793757735
resolution: 1.6, nmi: 0.5880930032778728
resolution: 1.7, nmi: 0.5880567298480635
resolution: 1.8, nmi: 0.5900292562910553
resolution: 1.9, nmi: 0.5941210955457285
resolution: 2.0, nmi: 0.5961544732652095
optimised clustering against batch
optimal cluster resolution: 1.0
optimal score: 0.596875005891779


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 46/46: 100%|██████| 46/46 [09:32<00:00, 11.74s/it, loss=1.26e+03, v_num=1]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=46` reached.


Epoch 46/46: 100%|██████| 46/46 [09:32<00:00, 12.45s/it, loss=1.26e+03, v_num=1]
resolution: 0.1, nmi: 0.8259786931999427
resolution: 0.2, nmi: 0.8469865471094599
resolution: 0.3, nmi: 0.8440896782366138
resolution: 0.4, nmi: 0.8439855716358041
resolution: 0.5, nmi: 0.839638258073822
resolution: 0.6, nmi: 0.8278219526371047
resolution: 0.7, nmi: 0.8172997404105852
resolution: 0.8, nmi: 0.8140146493548777
resolution: 0.9, nmi: 0.7944282930500546
resolution: 1.0, nmi: 0.7942607727760571


...

/home/kkovacs/.local/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:368: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  warnings.warn('The global conversion available with activate() '
/home/kkovacs/.local/lib/python3.8/site-packages/rpy2/robjects/numpy2ri.py:241: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  warnings.warn('The global conversion available with activate() '
/home/kkovacs/.local/lib/python3.8/site-packages/rpy2/robjects/conversion.py:28: DeprecationWarning: The use of converter in module rpy2.robjects.conversion is deprecated. Use rpy2.robjects.conversion.get_conversion() instead of rpy2.robjects.conversion.converter.
  warnings.warn(


Adding diffusion to step 4
Adding diffusion to step 5


/home/kkovacs/.local/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:368: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  warnings.warn('The global conversion available with activate() '
/home/kkovacs/.local/lib/python3.8/site-packages/rpy2/robjects/numpy2ri.py:241: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  warnings.warn('The global conversion available with activate() '
/home/kkovacs/.local/lib/python3.8/site-packages/rpy2/robjects/conversion.py:28: DeprecationWarning: The use of converter in module rpy2.robjects.conversion is deprecated. Use rpy2.robjects.conversion.get_conversion() instead of rpy2.robjects.conversion.converter.
  warnings.warn(
/home/kkovacs/.local/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:368: DeprecationWarning: The global convers

Migratory dendritic consists of a single batch or is too small. Skip.
Pericyte consists of a single batch or is too small. Skip.
SMC population consists of a single batch or is too small. Skip.
pDC consists of a single batch or is too small. Skip.
Blood cells consists of a single batch or is too small. Skip.
Small clust consists of a single batch or is too small. Skip.
cLISI score...
File has no entries. Doing nothing.
iLISI score...
File has no entries. Doing nothing.


AnnData object with n_obs × n_vars = 174977 × 2000
    obs: 'nCount_covid', 'nFeature_covid', 'orig.ident', 'Group', 'res.0', 'res.0.1', 'res.0.2', 'res.0.3', 'res.0.4', 'res.0.5', 'res.0.6', 'res.0.7', 'res.0.8', 'res.0.9', 'res.1', 'seurat_clusters', 'infection', 'cell_type', 'Mice', 'cell_label_short_0.6', 'nCount_covidneg', 'nFeature_covidneg', 'infection_neg', 'new_cell_type', 'study', 'CellTypes', 'CellTypes_short', 'samples', 'Timepoint', 'doublet', 'identifier.1', 'name', 'grouping', 'batch', 'cells', 'cluster', 'condition', 'age', 'sex', 'strain', 'treatment', 'tissue', 'time', 'platform', 'Tumor.or.healthy', 'Biological.replicate', 'Library', 'Barcode', 'Library.prep.batch', 'Total.counts', 'Percent.counts.from.mitochondrial.genes', 'Most.likely.Immgen.cell.type', 'Major.cell.type', 'Minor.subset', 'x', 'y', 'Geno', 'ID_old', 'hash.ID', 'SampleIDs', 'new.Celltypes', 'day', 'barcode', 'geo', 'predictions', 'clusters_1', 'clusters_2', 'clusters_3', 'n_genes_by_counts', 'total_c

AnnData object with n_obs × n_vars = 174977 × 43519
    obs: 'nCount_covid', 'nFeature_covid', 'orig.ident', 'Group', 'res.0', 'res.0.1', 'res.0.2', 'res.0.3', 'res.0.4', 'res.0.5', 'res.0.6', 'res.0.7', 'res.0.8', 'res.0.9', 'res.1', 'seurat_clusters', 'infection', 'cell_type', 'Mice', 'cell_label_short_0.6', 'nCount_covidneg', 'nFeature_covidneg', 'infection_neg', 'new_cell_type', 'study', 'CellTypes', 'CellTypes_short', 'samples', 'Timepoint', 'doublet', 'identifier.1', 'name', 'grouping', 'batch', 'cells', 'cluster', 'condition', 'age', 'sex', 'strain', 'treatment', 'tissue', 'time', 'platform', 'Tumor.or.healthy', 'Biological.replicate', 'Library', 'Barcode', 'Library.prep.batch', 'Total.counts', 'Percent.counts.from.mitochondrial.genes', 'Most.likely.Immgen.cell.type', 'Major.cell.type', 'Minor.subset', 'x', 'y', 'Geno', 'ID_old', 'hash.ID', 'SampleIDs', 'new.Celltypes', 'day', 'barcode', 'geo', 'predictions', 'clusters_1', 'clusters_2', 'clusters_3', 'n_genes_by_counts', 'total_

In [ ]:
runScib_SCANVI()

runScib_Non_raw_scanorama('scanorama')

runScib_Non_raw('mnn')

runScib_Non_raw('combat')

runScib_Non_raw('harmony')
